<a href="https://colab.research.google.com/github/subharks/Test/blob/main/WebScrapper_BigBasket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code to Extract data from Big Basket

***Prerequisites***

In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 904 kB 5.0 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.6 kB]
Hit:12 http://ppa.launchpad.net/

***Code to extract Data from Seller***

In [15]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys 
import time
import csv
from datetime import date
 
# Generate URL based on Search Item
def get_url(search_term):
  ## Generate URL based on Search Item
  template="https://www.bigbasket.com/cl/{}/?nc=nb"
  search_term=search_term.replace(' ','+')
 
  # add Seacrh term to URL
  url= template.format(search_term)
 
  # Append Page No
 # url += "#!page={}"
 
  return url
 
# Method to scroll the page till the end
def scroll_to_bottom(driver):
    SCROLL_PAUSE_TIME = 5
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
 
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
 
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
 
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
 
# Extract details of a given product
def extract_record(product):
  
  # Product Name
  try:
    productParentNode=product.find("div",{"qa":"product_name"})
    productName=productParentNode.a.text 
  except:
    productName="Missing"
    # grab the price 
 
  # Product Price
  try:
    priceParentNode=product.find("div",{"qa":"price"})
    price=priceParentNode.find("span",{"class","discnt-price"}).span.text.strip()
  except:
    price=0
 
  # Grab the Quantity 
  try:
    quantityParentNode=product.find('div','qnty-selection')
    quantityParentNodeBtn=quantityParentNode.button
    if quantityParentNodeBtn is None:
      quantity=quantityParentNode.find("span",{"ng-bind":"vm.selectedProduct.w"}).text.strip()
     # pricePerQuantity=0
    else:
      quantity=quantityParentNode.find("span",{"ng-bind":"product.w"}).text.strip()
      #pricePerQuantity=quantityParentNode.find("span",{"ng-bind":"product.sp"}).text.strip()
  except:
      quantity='Missing'
 
 
  today = date.today()
  date_published = today.strftime("%Y-%m-%d")
  result = (date_published,productName, quantity,price)
  return result
 
 
def generate_filename(filenm):
    today=date.today()
    timestamp = today.strftime("%Y%m%d")
    filename = filenm + '_' + timestamp + '.csv'
  #  filename="/content/drive/MyDrive/ML_Data/BigBasket/" +  filenm + '_' + timestamp + '.csv'
    return filename

 

# Method to set properties of chrome driver
#The Chromeoptions Class is a concept in Selenium WebDriver for manipulating various properties of the Chrome driver.
def create_web_driver():
  from selenium import webdriver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument("user-agent=xxx")
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  return driver
 
 
#fruits-vegetables
def main(search_term):
  """Run main Program Routine""" 
  filename = generate_filename(search_term)
  records=[]
 
  #driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver=create_web_driver()
  #url=get_url(search_term)
  url="https://www.bigbasket.com/cl/fruits-vegetables/?nc=nb"
  #time.sleep(10)
  driver.get(url)
  scroll_to_bottom(driver)
  page_soup=BeautifulSoup(driver.page_source,"html.parser")
  time.sleep(10)

   # Extract all the products
  products=page_soup.find_all("div",{"qa":"product"})
      
  print("products=",len(products))
  for product in products:
    record=extract_record(product)
    if record:
      records.append(record)
  driver.close()  
 
  with open(generate_filename("BigBasket"), 'w', newline='', encoding='utf-8') as f:
     writer = csv.writer(f)
     writer.writerow(["Date","Product Description", "Quantity", "Price"])
     writer.writerows(records)

Main code to call

In [16]:

#driver.close()
records=main("fruits-vegetables")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: use options instead of chrome_options


products= 192
